<a href="https://colab.research.google.com/github/Danish-2003/DataPreProcessingWithPython/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataCleaning/marketing_campaign_data_messy.csv')

In [ ]:
print(f"Loaded Data : {df.shape[0]} row , {df.shape[1] }columns")

Loaded Data : 2020 row , 12columns


# **`Header cleaning`**

In [35]:

print(df.columns.tolist())

df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')
print("Fix Applied ")
print(df.columns.tolist())

[' Campaign_ID ', 'Campaign_Name', 'Start_Date', 'End_Date', 'Channel', 'Impressions', 'Clicks ', 'Spend', 'Conversions', 'Active', 'Clicks', 'Campaign_Tag']
Fix Applied 
['campaign_id', 'campaign_name', 'start_date', 'end_date', 'channel', 'impressions', 'clicks', 'spend', 'conversions', 'active', 'clicks', 'campaign_tag']


In [36]:
df.head(2)

,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,clicks,campaign_tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24 00:00:00,2023-12-13,TikTok,16795,197,$102.82,20.0,Y,NaN,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06 00:00:00,2023-05-12,Facebook,1860,30,24.33,1.0,0,NaN,FA


# **`Type conversion and currency changing`**

In [37]:

dirty_spend_mask = df['spend'].astype(str).str.contains(r'\$')
print(df.loc[dirty_spend_mask,['campaign_id','spend']].head())
df['spend'] = df['spend'].astype(str).str.replace(r'[^\d.-]','',regex = True)
df['spend'] = pd.to_numeric(df['spend'],errors = 'coerce')

print('Fix Applied !!')
print(df.loc[dirty_spend_mask,['campaign_id','spend']].head(3))

   campaign_id     spend
0    CMP-00001   $102.82
21   CMP-00022   $2428.4
22   CMP-00023  $4726.22
31   CMP-00032  $2759.35
32   CMP-00033  $2393.02
Fix Applied !!
   campaign_id    spend
0    CMP-00001   102.82
21   CMP-00022  2428.40
22   CMP-00023  4726.22


# **`Categorical Typos`**

In [ ]:

df['channel'].unique()

array(['TikTok', 'Facebook', 'Email', 'Instagram', 'Google Ads', 'E-mail',
       nan, 'Gogle', 'Tik_Tok', 'Facebok', 'Insta_gram'], dtype=object)

In [38]:
channel_map = {
    'Tik_Tok' : 'TikTok',
    'Insta_gram' : 'Instagram',
    'Facebok':'Facebook',
    'Gogle':'Google Ads',
    'E-mail':'Email',
    'N/A':np.nan,
}

In [39]:
df['channel'] = df['channel'].replace(channel_map)

In [40]:
df['channel'].unique()

array(['TikTok', 'Facebook', 'Email', 'Instagram', 'Google Ads', nan],
      dtype=object)

# **`Handling Mixed Booleans`**




In [41]:
df['active'].unique()

array(['Y', '0', 'No', 'True', 'Yes', '1', 'False'], dtype=object)

In [42]:
bool_map = {
    'Y' : True,
    'Yes': True,
    'No': False,
    '1' : True,
    '0' : False,
}

df['active'] = df['active'].map(bool_map).fillna(False).astype(bool)
df['active'].unique()

/tmp/ipython-input-2962016668.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['active'] = df['active'].map(bool_map).fillna(False).astype(bool)


array([ True, False])

# **`Date Pasrsing `**

In [43]:
df['start_date'] = pd.to_datetime(df['start_date'],errors = 'coerce')
df['end_date'] = pd.to_datetime(df['end_date'],dayfirst=True ,errors = 'coerce')


/tmp/ipython-input-630165433.py:2: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['end_date'] = pd.to_datetime(df['end_date'],dayfirst=True ,errors = 'coerce')


# **`Logical Integrity (Click VS Impression )`**

In [44]:
df = df.loc[:, ~df.columns.duplicated()]


In [45]:
impossible_mask = df['clicks'] > df['impressions']
print(df.loc[impossible_mask,['campaign_id','clicks','impressions']].head(3))

Empty DataFrame
Columns: [campaign_id, clicks, impressions]
Index: []


# **`Logical Integrity (Time Mismatch)`**

---



In [46]:
time_travel_mask = df['start_date'] > df['end_date']

In [47]:
print(df.loc[time_travel_mask,['campaign_id','start_date','end_date']].head(3))
df.loc[time_travel_mask,'end_date'] = df.loc[time_travel_mask,'start_date']+pd.Timedelta(days=30)
print('Fix Applied')
print(df.loc[time_travel_mask,['campaign_id','start_date','end_date']].head(3))


   campaign_id start_date   end_date
23   CMP-00024 2023-05-06 2023-05-01
54   CMP-00055 2023-09-01 2023-08-27
71   CMP-00072 2023-02-01 2023-01-27
Fix Applied
   campaign_id start_date   end_date
23   CMP-00024 2023-05-06 2023-06-05
54   CMP-00055 2023-09-01 2023-10-01
71   CMP-00072 2023-02-01 2023-03-03


# **`Handling Outliers ( Winsorizing)`**

In [48]:
Q1 = df['spend'].quantile(0.25)
Q3 = df['spend'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q3 - Q1
upper_bound = Q3 + (3 * IQR)

outlier_mask = df['spend'] > upper_bound
df.loc[outlier_mask,['campaign_id','spend']].head(2)

df.loc[outlier_mask,'spend'] = upper_bound

# **`String Parsing (Feature Extraction)`**

In [49]:
print(df['campaign_name'].head(3))

0    Q4_Summer_CMP-00001
1    Q1_Launch_CMP-00002
2    Q3_Winter_CMP-00003
Name: campaign_name, dtype: object


In [50]:
df['season'] = df['campaign_name'].str.extract(r'Q\d_([^_]+)_')

In [ ]:
df.head(2)

,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,campaign_tag,season
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24,2023-12-13,TikTok,16795,197,102.82,20.0,True,TI,Summer
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06,2023-05-12,Facebook,1860,30,24.33,1.0,False,FA,Launch
